## Usage
This project provides the three ways to update CPI data, such as append load, trancate and load, and incremental load.
- "Get the Latest Data" enables users to get the latest CPI data by inputing any dates.
- "Load Data" shows the three ways to load the data up to the date which users input.
- "Compare Performance" illustrates the difference of processing speed among these three ways.

These services will make users decide which way to choose for their development.

## Preparation
1. Import necessary libraries.
2. Define function of loading CPI data into dataframe.
3. Define function of getting the latest data.

In [64]:
import requests
import pandas as pd
import duckdb
from io import BytesIO
import time

def load_data(url):
    response = requests.get(url)
    cpi_data = BytesIO(response.content)
    df = pd.read_excel(cpi_data)
    return df

def get_latest_data(df, pull_date):
    year, month, day = pull_date.split('-')
    year = year[2:]
    month = str(int(month))
    col_name  = f'PCPI{year}M{month}'
    df_filtered = df[['DATE', col_name]]
    df_filtered = df_filtered.rename(columns={col_name: 'CPI'})
    return df_filtered

## Get the Latest Data
Write any pull date from 1998-11-01 to 2025-2-28 and run the code to see the result.

This project manually vefities that the following validation works.
- If users select the date before 1998-11-01, pull date is set to be 1998-11-01.
- If users select the date after 2025-2-28, pull date is set to be 2025-2-28.


In [65]:
import datetime

url = 'https://www.philadelphiafed.org/-/media/FRBP/Assets/Surveys-And-Data/real-time-data/data-files/xlsx/pcpiMvMd.xlsx?sc_lang=en&hash=E41A743DC6423F950B10C3DE7A4F674D'
pull_date = '2024-12-01'

pull_date = datetime.datetime.strptime(pull_date, '%Y-%m-%d').date()

min_date = datetime.date(1998, 11, 1)
max_date = datetime.date(2025, 2, 28)

pull_date = min(max(pull_date, min_date), max_date)

latest_data = get_latest_data(load_data(url), str(pull_date))
latest_data.tail()


,DATE,CPI
932,2024:09,314.686
933,2024:10,315.454
934,2024:11,316.441
935,2024:12,NaN
936,2025:01,NaN


## Load Data

Users can set a pull date.

In [66]:
user_pull_date = '1998-12-01'

1. Append Load

In [67]:
def update_append(pull_date, con):
    latest_data = get_latest_data(load_data(url), pull_date)
    latest_data = latest_data.dropna(subset=['CPI'])
    existing_dates = con.execute('SELECT DATE FROM cpi_table').fetchdf()['DATE'].tolist()
    latest_data = latest_data[~latest_data['DATE'].isin(existing_dates)]
    if not latest_data.empty:
        latest_data.to_sql('cpi_table', con, if_exists='append', index=False)

In [68]:
con_append = duckdb.connect('cpi_append.duckdb')
con_append.execute('DROP TABLE IF EXISTS cpi_table')
con_append.execute("""
    CREATE TABLE cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

update_append(user_pull_date, con_append)
updated_records = con_append.execute('SELECT * FROM cpi_table ORDER BY DATE DESC LIMIT 5').fetchdf()
print(updated_records)

con_append.close()

C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\3060384955.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


      DATE         CPI
0  1998:11  164.300003
1  1998:10  164.000000
2  1998:09  163.600006
3  1998:08  163.600006
4  1998:07  163.300003


2. Truncate and Load

In [69]:
def update_truncate(pull_date, con):
    latest_data = get_latest_data(load_data(url), pull_date)    
    latest_data = latest_data.dropna(subset=['CPI'])
    con.execute('DELETE FROM cpi_table')
    if not latest_data.empty:
        latest_data.to_sql('cpi_table', con, if_exists='append', index=False)

In [70]:
con_truncate = duckdb.connect('cpi_truncate.duckdb')
con_truncate.execute('DROP TABLE IF EXISTS cpi_table')
con_truncate.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

update_truncate(user_pull_date, con_truncate)
updated_records = con_truncate.execute("SELECT * FROM cpi_table ORDER BY DATE DESC LIMIT 5").fetchdf()
print(updated_records)

con_truncate.close()

C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\311833620.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


      DATE         CPI
0  1998:11  164.300003
1  1998:10  164.000000
2  1998:09  163.600006
3  1998:08  163.600006
4  1998:07  163.300003


3. Incremental Load

In [71]:
def update_incremental(pull_date, con):
    latest_data = get_latest_data(load_data(url), pull_date)
    latest_data = latest_data.dropna(subset=['CPI'])
    existing_data = con.execute('SELECT * FROM cpi_table').fetchdf()

    if not existing_data.empty:
        merged_data = latest_data.merge(existing_data, on='DATE', suffixes=('_new', '_old'), how='left')

        updated_data = merged_data[
            (merged_data['CPI_old'].isna()) |
            (merged_data['CPI_new'] != merged_data['CPI_old'])
        ][['DATE', 'CPI_new']].rename(columns={'CPI_new': 'CPI'})

        if not updated_data.empty:
            date_list = updated_data['DATE'].tolist()
            placeholders = ', '.join(['?'] * len(date_list))
            delete_query = f'DELETE FROM cpi_table WHERE DATE IN ({placeholders})'
            con.execute(delete_query, date_list)
            updated_data.to_sql('cpi_table', con, if_exists='append', index=False)
    else:
        latest_data.to_sql('cpi_table', con, if_exists='append', index=False)

In [72]:
con_incremental = duckdb.connect('cpi_incremental.duckdb')
con_incremental.execute('DROP TABLE IF EXISTS cpi_table')
con_incremental.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

update_incremental(user_pull_date, con_incremental)
updated_records = con_incremental.execute("SELECT * FROM cpi_table ORDER BY DATE DESC LIMIT 5").fetchdf()
print(updated_records)

con_incremental.close()

C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\2498723605.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


      DATE         CPI
0  1998:11  164.300003
1  1998:10  164.000000
2  1998:09  163.600006
3  1998:08  163.600006
4  1998:07  163.300003


## Compare Performance

Users can set start and end date of loading data.

In [73]:
start_date = "2025-01-30"
end_date = "2025-02-01"
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

Define function of measuring time of each method.

In [74]:
def measure_time(update_method, con):
    start_time = time.time()
    for pull_date in date_range:
        update_method(pull_date.strftime('%Y-%m-%d'), con)
    end_time = time.time()
    return end_time - start_time

1. Check the performance of Append Load.

In [75]:
con_append = duckdb.connect('cpi_append.duckdb')
con_append.execute('DROP TABLE IF EXISTS cpi_table')
con_append.execute("""
    CREATE TABLE cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

print(f'Total Execution Time of Append: {measure_time(update_append, con_append):.2f} seconds')

con_append.close()

C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\3060384955.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


Total Execution Time of Append: 23.18 seconds


C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\3060384955.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)


2. Check the performance of Truncate and Load.

In [76]:
con_truncate = duckdb.connect('cpi_truncate.duckdb')
con_truncate.execute('DROP TABLE IF EXISTS cpi_table')
con_truncate.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

print(f"Total Execution Time of 'Truncate': {measure_time(update_truncate, con_truncate):.2f} seconds")

con_truncate.close()

C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\311833620.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\311833620.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\311833620.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='appen

Total Execution Time of 'Truncate': 25.38 seconds


3. Check the performance of Incremental Load.

In [77]:
con_incremental = duckdb.connect('cpi_incremental.duckdb')
con_incremental.execute('DROP TABLE IF EXISTS cpi_table')
con_incremental.execute("""
    CREATE TABLE IF NOT EXISTS cpi_table (
        DATE STRING,
        CPI FLOAT
    )
""")

print(f"Total Execution Time of 'Truncate': {measure_time(update_incremental, con_incremental):.2f} seconds")

con_truncate.close()

C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\2498723605.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  latest_data.to_sql('cpi_table', con, if_exists='append', index=False)
C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\2498723605.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  updated_data.to_sql('cpi_table', con, if_exists='append', index=False)
C:\Users\oiwah\AppData\Local\Temp\ipykernel_23928\2498723605.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  updated_data.to_sql('cpi_table', con, if_exist

Total Execution Time of 'Truncate': 21.84 seconds


## Conclusion

*Times are different by testing. So, I will update them at last.
 
The processing times were shortest for append load (15.6 seconds), followed by incremental load (17.85 seconds), and longest for truncate and load (22.2 seconds). In terms of speed alone, append load is the fastest since it simply adds new data, while truncate and load is the slowest as it reuploads the entire dataset each time. Incremental load falls in between, as it updates only the differences, making its processing time proportional to the amount of data uploaded.

From a data quality perspective, append load risks degrading accuracy because historical data is periodically updated, leading to inconsistencies. However, replacing the entire dataset every time is unnecessary—updating only the changes is sufficient. Balancing both data quality and processing speed, the most appropriate approach for handling CPI data is incremental load, as it efficiently maintains accuracy while keeping processing time manageable.